In [1]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import model_selection, ensemble
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb



/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# RANDOM FOREST

def random_forest(x1, y1, x2, y2, n_estimators=20, max_features=5):
    rf = ensemble.RandomForestClassifier(n_estimators=n_estimators, max_features=max_features)
    rf.fit(x1, y1)
    err_train = np.mean(y1 != rf.predict(x1))
    err_test  = np.mean(y2  != rf.predict(x2))
    return err_train, err_test

# GRAD BOOST

def grad_boost(x1, y1, x2, y2, n_estimators=20):
    gbt = ensemble.GradientBoostingClassifier(n_estimators=n_estimators)
    gbt.fit(x1, y1)
    err_train = np.mean(y1 != gbt.predict(x1))
    err_test = np.mean(y2 != gbt.predict(x2))
    return err_train, err_test

# XGBOOST

def xgboost(x1, y1, x2, y2):
    model = xgb.XGBClassifier()
    model.fit(x1, y1)
    err_train = np.mean(y1 != model.predict(x1))
    err_test = np.mean(y2 != model.predict(x2))
    return err_train, err_test


In [2]:
xytte = pd.read_csv("data/xytte.csv")

####################### ВНИМАНИЕ #######################
# здесь трейн это которые в задаче трейн, с ответами
# тест без ответов
# но дальше в алгоритмах трейн будет разбиваться на два подсета
# которые будут у меня называться x1-y1 и x2-y2, а по смыслу как раз трейн и тест
# надо не перепутать

x_train = xytte[xytte.returned == xytte.returned].reset_index().drop("returned", axis=1)
y_train = xytte[xytte.returned == xytte.returned].reset_index()[["returned"]]
x_test  = xytte[xytte.returned != xytte.returned].reset_index().drop("returned", axis=1)

In [3]:
# проверяем, что нигде не налажали
# читаем из файла оригинальные данные

x_test_ref = pd.read_csv("data/x_test.csv", sep=";", dtype=np.float32)
x_train_ref = pd.read_csv("data/x_train.csv", sep=";", dtype=np.float32)
y_train_ref = pd.read_csv("data/y_train.csv", sep=";", dtype=np.float32, header=None, names=["returned"])

# заводим функцию на сравнение таблиц

def df_equal(df1, df2):
    if list(df1.columns) != list(df2.columns): 
        print("Mismatch columns:")
        print("df1: ", list(df1.columns))
        print("df2: ", list(df2.columns))
        return False
    if list(df1.index) != list(df2.index): 
        print("Mismatch index:")
        print("df1: ", list(df1.columns)[0], ":", list(df1.columns)[-1])
        print("df2: ", list(df2.columns)[0], ":", list(df2.columns)[-1])
        return False
    
    ne_stacked = (df1 != df2).stack()
    changed = ne_stacked[ne_stacked]
    changed.index.names = ['id', 'col']

    difference_locations = np.where(df1 != df2)
    changed_from = df1.values[difference_locations]
    changed_to = df2.values[difference_locations]
    diff = pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
    return not diff[diff["from"] - diff["to"] > 0.000001].count()[0]

# ассертим наши таблицы

assert(df_equal(x_test[x_test_ref.columns], x_test_ref))
assert(df_equal(x_train[x_train_ref.columns], x_train_ref))
assert(df_equal(y_train, y_train_ref))

In [21]:
params_orig = [
'maxPlayerLevel',
'numberOfAttemptedLevels',
'attemptsOnTheHighestLevel',
'totalNumOfAttempts',
'averageNumOfTurnsPerCompletedLevel',
'numberOfBoostersUsed',
'fractionOfUsefullBoosters',
'totalScore',
'totalBonusScore',
'totalStarsCount',
'numberOfDaysActuallyPlayed',
]
     
params_mod = [
#'attemptsOnTheHighestLevel_dvd',
'attemptsOnTheHighestLevel_ln',
'attemptsPerDay',
'averageNumOfTurnsPerCompletedLevel_dvd',
'maxPlayerLevel_ln',
'numberOfAttemptedLevels_dvd',
#'numberOfBoostersUsed_dvd',
'numberOfBoostersUsed_dvd_ln',
#'numberOfBoostersUsed_ln',
'numberOfDaysActuallyPlayed_ln',
'totalBonusScore_dvd',
'totalNumOfAttempts_ln',
#'totalScore_ln',
'totalScore_ln_dvd',
#'totalStarsCount_dvd',
'totalStarsCount_dvd_ln',
#'totalStarsCount_ln',
]

params_bool = [
'allAttemptsOnTheHighestLevel',
'attLevelsMoreThanMaxLevel',
'doReturnOnLowerLevels',
'zeroTotalScore',
'zeroTurnsPerCompletedLevel'
]

params_orig_norm = [
'attemptsOnTheHighestLevel_norm',
'attemptsPerDay_norm',
#'numberOfBoostersUsed_norm',
#'maxPlayerLevel_norm',
'fractionOfUsefullBoosters_norm',
#'averageNumOfTurnsPerCompletedLevel_norm',
'numberOfDaysActuallyPlayed_norm',
'totalNumOfAttempts_norm',
'totalBonusScore_norm',
'totalScore_norm',
'totalStarsCount_norm',
]

params_mod_norm = [
'attemptsOnTheHighestLevel_dvd_norm',
#'attemptsOnTheHighestLevel_ln_norm',
'averageNumOfTurnsPerCompletedLevel_dvd_norm',
'maxPlayerLevel_ln_norm',
'numberOfAttemptedLevels_dvd_norm',
#'numberOfAttemptedLevels_norm',
'numberOfBoostersUsed_dvd_ln_norm',
#'numberOfBoostersUsed_dvd_norm',
#'numberOfBoostersUsed_ln_norm',
'numberOfDaysActuallyPlayed_ln_norm',
'totalBonusScore_dvd_norm',
'totalNumOfAttempts_ln_norm',
'totalScore_ln_dvd_norm',
#'totalScore_ln_norm',
'totalStarsCount_dvd_ln_norm',
#'totalStarsCount_dvd_norm',
'totalStarsCount_ln_norm',
]

In [22]:
#params = params_orig + ["doReturnOnLowerLevels"]
#params = params_mod + params_bool
params = params_orig_norm

In [23]:
x_train_feed = x_train[params]
x_test_feed = x_test[params]
y_train_feed = list(y_train.returned)
#y_test_feed = list(y_test)

x1, x2, y1, y2 = train_test_split(x_train_feed, y_train_feed, test_size = 0.3)

In [24]:
print(random_forest(x1, y1, x2, y2))
print(grad_boost(x1, y1, x2, y2))
print(xgboost(x1, y1, x2, y2))

(0.013614280872217829, 0.19282984051667326)
(0.16964184837871427, 0.17279557137208382)
(0.16715625353067451, 0.17411361539475417)


In [7]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, params[idx], importances[idx]))

Feature importances:
 1. feature 'numberOfDaysActuallyPlayed' (0.2684)
 2. feature 'totalNumOfAttempts' (0.1543)
 3. feature 'averageNumOfTurnsPerCompletedLevel' (0.1088)
 4. feature 'totalScore' (0.1074)
 5. feature 'maxPlayerLevel' (0.0861)
 6. feature 'totalBonusScore' (0.0633)
 7. feature 'numberOfBoostersUsed' (0.0480)
 8. feature 'totalStarsCount' (0.0465)
 9. feature 'fractionOfUsefullBoosters' (0.0434)
10. feature 'attemptsOnTheHighestLevel' (0.0418)
11. feature 'numberOfAttemptedLevels' (0.0245)
12. feature 'doReturnOnLowerLevels' (0.0075)
